In [1]:
import uniform_model
import simulation
import diagnostics
import phare

# Running miniphare from Python Tutorial

First create a simulation object. Note that the 'path' is relative to where the script will be executed

In [2]:
simu = simulation.Simulation(time_step_nbr=1000, boundary_types="periodic",
                             cells=80, dl=0.1, final_time=1.,path='simu1')

Then define some diagnostics. We create:
- two FluidDiagnostics : $\rho_s$ and $\phi_s$ for species 'proton1'
- two ElectromagDiagnostics : for $\mathbf{E}$ and $\mathbf{B}$

In [3]:
fd1 = diagnostics.FluidDiagnostics(name="FluidDiagnostics1", diag_type="rho_s",
                                   write_every=10, compute_every=5, start_iteration=0,
                                   last_iteration=1000, species_name="proton1")

fd2 = diagnostics.FluidDiagnostics(name="FluidDiagnostics2",
                                   diag_type="flux_s", write_every=10, compute_every=5,
                                   start_iteration=0, last_iteration=1000, species_name="proton1")

ed1 = diagnostics.ElectromagDiagnostics(name="ElectromagDiagnostics1", diag_type="E",
                                        write_every=10, compute_evert=5, start_teration=0, last_iteration=1000)

ed2 = diagnostics.ElectromagDiagnostics(name="ElectromagDiagnostics2", diag_type="B",
                                        write_every=10, compute_evert=5, start_teration=0, last_iteration=1000)

add the diagnostics to the simulation

In [4]:
simu.add_diagnostics(fd1)
simu.add_diagnostics(fd2)
simu.add_diagnostics(ed1)
simu.add_diagnostics(ed2)

RuntimeError: Error: invalid diagnostics duration

Now make a model and set the simulation initial condition to this model.
Here we create a `UniformModel`.
We add default fields (0 electric field and constant unit magnetic field are default)
and add 2 proton species. The first one is defined as default (no bulk velocity, unit density, etc.) the second has a custom constant density of twice that of the first species

In [8]:
model = uniform_model.UniformModel()

model.add_fields()
model.add_species("proton1")
model.add_species("proton2", density=2.)

simu.set_model(model)

Then we prepare a job, this will write an INI file where it should be (simu.path == 'simu1' in this case)
This will create the job directory under the specified path, and within it the diagnostics directories.

In [9]:
simulation.prepare_job(simu)

preparing job...
mkdir simu1
writing ini file in simu1/
mkdir simu1/fluiddiagnostics1
mkdir simu1/fluiddiagnostics2
mkdir simu1/electromagdiagnostics1
mkdir simu1/electromagdiagnostics2


In [ ]:
phare.MiniPHARE().run(simu)

and finally run the code with this simulation.
This requires 'miniphare' executable to be in your PATH environment variable.

Alternatively, if the directories and INI files already have been generated, one can also use the convenience executable phare.py.

`./phare.py path_to_job_directory`

Provided your 'miniphare' executable is in the PATH environment variable, this command will jump into 'path_to_job_directory' (where 'phare.ini' and diagnostics directories are) and run 'miniphare phare.ini' for you.

# Re-using simulation objects to run parametric studies

One can easily re-use a simulation object by copying it.
Once copied, one can make a list and run them one after this other.
The following example copies the above reference simulation and modifies it to add an Oxygen population for instance.

In [14]:
import copy

In [16]:
simu2 = copy.copy(simu)
simu2.model.add_species("oxygen", density=0.1, mass=16)
simu2.path = 'simu2'

my_simulations = [simu, simu2]

for sim in my_simulations:
    phare.MiniPHARE().run(sim)
    